In [1]:
import datasets
import polars as pl

from hashlib import sha256

In [2]:
data = pl.read_parquet("poki.pq").sort("date")
data = data.with_columns(pl.Series(range(len(data))).alias("id"))
data = data[["id", *data.columns[:-1]]]
data

id,title,description,authors,proofreaders,date,date-precision,tags,original,license,sources,archives,preprocessing,accessibility-notes,notes,text
i64,str,str,list[str],list[str],str,str,list[str],struct[2],str,list[str],list[str],str,str,str,str
0,"""lawa monsuta""",null,"[""jan Juli""]",null,"""2001-01-01""","""none""",null,null,"""CC-BY-SA 4.0""","[""https://github.com/lipukule/site/blob/main/content/post/2023-04-14-lawa-monsuta.md"", ""https://lipukule.org/post/2023/04/10/lawa-monsuta/""]",null,null,null,null,"""o sona a: moli en pilin ike li…"
1,"""jan en soweli""",null,"[""Michael F.""]",null,"""2001-01-01""","""none""","[""original work"", ""poetry""]",null,"""CC BY-NC 3.0""","[""http://failbluedot.com/toki_pona/jan_en_soweli""]","[""https://web.archive.org/web/20140305064458/http://failbluedot.com/toki_pona/jan_en_soweli""]",null,null,null,"""jan en soweli jan mute li pili…"
2,"""Dixie""",null,"[""jan Pije""]",null,"""2001-01-01""","""none""","[""translation"", ""song"", … ""confederacy""]","{""Dixie"",[""unknown""]}","""CC0 1.0""","[""http://tokipona.net/tp/janpije/dixie.php""]","[""https://web.archive.org/web/20151003033703/http://tokipona.net/tp/janpije/dixie.php""]",null,null,null,""" mi wile lon ma pi kasi len. …"
3,"""kasi nasa en telo nasa""",null,"[""Michael F.""]",null,"""2001-01-01""","""none""","[""original work"", ""poetry""]",null,"""CC BY-NC 3.0""","[""http://failbluedot.com/toki_pona/kasi_nasa_telo_nasa""]","[""https://web.archive.org/web/20140305061010/http://failbluedot.com/toki_pona/kasi_nasa_telo_nasa""]",null,null,null,"""kasi nasa en telo nasa telo na…"
4,"""jan lawa""",null,"[""Michael F.""]",null,"""2001-01-01""","""none""","[""original work"", ""poetry""]",null,"""CC BY-NC 3.0""","[""http://failbluedot.com/toki_pona/jan_lawa""]","[""https://web.archive.org/web/20140305061518/http://failbluedot.com/toki_pona/jan_lawa""]",null,null,null,"""jan lawa mi pona ala e sina o …"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1497,"""sina wile pali e musi pi sitel…",null,"[""kije Weku""]",null,"""2025-08-28""","""day""","[""sona""]",null,"""CC BY-SA 4.0""","[""https://liputenpo.org/lipu/nanpa-pilin/"", ""https://liputenpo.org/pdfs/0033pilin.pdf"", ""https://github.com/lipu-tenpo/liputenpo.org/tree/main/toki""]",null,null,null,null,"""sina wile pali mute la, sina k…"
1498,"""lipu suno pi jan moli""","""A diary of a dead person.""","[""jan Sepo""]",null,"""2025-09-03""","""day""","[""diary"", ""death"", … ""monsuta""]",null,"""CC BY-SA 4.0""","[""https://discord.com/channels/301377942062366741/340307145373253642/1412768840412303441""]",null,null,null,"""Created on September 3, 2025, …","""lipu suno pi jan moli mi wile …"
1499,"""sona sin pona tan jan sewi Joj…","""A revised translation of John'…","[""Jonathan Hill"", ""jan Tene""]",null,"""2025-09-07""","""day""","[""religion"", ""nasin sewi"", … ""story""]","{""The Gospel of John"",null}","""CC BY-SA 4.0""","[""https://denesszanto.itch.io/lipu-sewi""]",null,null,null,null,"""sona sin pona tan jan sewi Joj…"


In [3]:
data = data.with_columns(
    data["text"]
        .map_elements(
            lambda t:
            int(sha256(t.encode()).hexdigest(), 16) % 10000
        )
        .alias("hash")
)
data["hash"].plot.hist()

alt.Chart(...)

In [4]:
dataset_dict = datasets.DatasetDict()
dataset_dict["train"] = datasets.Dataset.from_polars(data.filter(data["hash"] > 500)[data.columns[:-1]])
dataset_dict["validation"] = datasets.Dataset.from_polars(data.filter(data["hash"] <= 500)[data.columns[:-1]])

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'description', 'authors', 'proofreaders', 'date', 'date-precision', 'tags', 'original', 'license', 'sources', 'archives', 'preprocessing', 'accessibility-notes', 'notes', 'text'],
        num_rows: 1433
    })
    validation: Dataset({
        features: ['id', 'title', 'description', 'authors', 'proofreaders', 'date', 'date-precision', 'tags', 'original', 'license', 'sources', 'archives', 'preprocessing', 'accessibility-notes', 'notes', 'text'],
        num_rows: 69
    })
})

In [5]:
dataset_dict.push_to_hub("NetherQuartz/poki-lapo")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/NetherQuartz/poki-lapo/commit/c914dc4bff13c06676ac0d1659a8874d0b034f58', commit_message='Upload dataset', commit_description='', oid='c914dc4bff13c06676ac0d1659a8874d0b034f58', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/NetherQuartz/poki-lapo', endpoint='https://huggingface.co', repo_type='dataset', repo_id='NetherQuartz/poki-lapo'), pr_revision=None, pr_num=None)